In [ ]:
import os 

os.environ['GOOGLE_API_KEY'] = ""

In [ ]:
! pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate


### Indexing

##### Document Loader (Ingestion)

In [7]:
video_id = "Gfr50f6ZBvo"
try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id,languages= ['en'])
    
    transcript = "".join(chunk['text'] for chunk in transcript_list)
    print(transcript)
except TranscriptsDisabled:
    print("No caption available for these video")
    

the following is a conversation withdemus hasabisceo and co-founder of deepminda company that has published and buildssome of the most incredible artificialintelligence systems in the history ofcomputing including alfred zero thatlearnedall by itself to play the game of goldbetter than any human in the world andalpha fold two that solved proteinfoldingboth tasks considered nearly impossiblefor a very long timedemus is widely considered to be one ofthe most brilliant and impactful humansin the history of artificialintelligence and science and engineeringin generalthis was truly an honor and a pleasurefor me to finally sit down with him forthis conversation and i'm sure we willtalk many times again in the futurethis is the lex friedman podcast tosupport it please check out our sponsorsin the description and now dear friendshere's demishassabislet's start with a bit of a personalquestionam i an ai program you wrote tointerview people until i get good enoughto interview youwell i'll be imp

In [8]:
transcript_list

[{'text': 'the following is a conversation with',
  'start': 0.08,
  'duration': 3.44},
 {'text': 'demus hasabis', 'start': 1.76, 'duration': 4.96},
 {'text': 'ceo and co-founder of deepmind', 'start': 3.52, 'duration': 5.119},
 {'text': 'a company that has published and builds',
  'start': 6.72,
  'duration': 4.48},
 {'text': 'some of the most incredible artificial',
  'start': 8.639,
  'duration': 4.561},
 {'text': 'intelligence systems in the history of',
  'start': 11.2,
  'duration': 4.8},
 {'text': 'computing including alfred zero that',
  'start': 13.2,
  'duration': 3.68},
 {'text': 'learned', 'start': 16.0, 'duration': 2.96},
 {'text': 'all by itself to play the game of gold',
  'start': 16.88,
  'duration': 4.559},
 {'text': 'better than any human in the world and',
  'start': 18.96,
  'duration': 5.6},
 {'text': 'alpha fold two that solved protein',
  'start': 21.439,
  'duration': 4.241},
 {'text': 'folding', 'start': 24.56, 'duration': 4.16},
 {'text': 'both tasks consider

###### Text Splitter

In [9]:
spliter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap = 200)
chunks = spliter.create_documents([transcript])



In [10]:
len(chunks)

163

###### Embedding 

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

###### Vector_Store

In [12]:
vector_store = FAISS.from_documents(chunks,embeddings)


In [13]:
vector_store

In [14]:
vector_store.index_to_docstore_id

{0: 'a0772fd4-c309-4892-9e1c-8e6bca862ce9',
 1: '93014b48-9c91-4bec-95cb-316246ed5e81',
 2: 'fe2c6cbf-033f-47da-85bd-e7ac755062de',
 3: 'b89040a0-394e-44d5-8eb7-2a671cba60fb',
 4: 'fa5266e2-5bca-4e53-82fc-2f02b98d8f9d',
 5: 'ac1b1605-78c1-4378-9ed8-4e808e0622f4',
 6: 'd2f731dc-9daf-4ae0-bfe8-166062322d00',
 7: '447a1c46-57d7-47d0-99f6-90e68deadcfa',
 8: '2cbf9905-1a22-4fd2-a8b3-abee3eb10a32',
 9: 'bf0389f4-6680-481f-ba2f-943776ea292f',
 10: '4b21ad18-5d9d-4424-95d0-9818ebfc2ab8',
 11: '3dbcc537-6075-4d61-a0fb-9fa5a91411c6',
 12: '4b1a7e63-2261-48a1-9567-80a6d7b87312',
 13: 'e52f3068-12f0-4081-b3b7-1678f047259f',
 14: 'e9618fbc-fa29-40c7-89d5-875f32e7a4d8',
 15: '483ae1a3-f0ed-4d45-9bb1-521c6b0ca13a',
 16: 'a1a79fec-c571-47dd-b712-7c98728692e3',
 17: '452402dc-662e-4408-8b97-d394a70b728f',
 18: 'a80d810c-82c7-471d-ad3b-bb8c399e39ac',
 19: 'd67a075f-c4ed-4596-8344-c384e1b6488f',
 20: '1d2cbd0d-14b0-44dd-9762-db559c173988',
 21: 'a53f8417-7506-45b1-aee1-54393f4631e5',
 22: 'a64cde30-651b-

In [15]:
vector_store.get_by_ids(['3e06234a-fe3e-499b-be77-159858dd0144'])

[]

#### Retriever

In [56]:
retriever = vector_store.as_retriever(search_type= "similarity",search_kwargs= {"k":3})

In [17]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FE146B6950>, search_kwargs={'k': 3})

In [18]:
retriever = retriever.invoke('What is Artificial Intelligence')
retriever

[Document(id='a1a79fec-c571-47dd-b712-7c98728692e3', metadata={}, page_content="had alphabeta search all of that and that was myfirst ai program i remember that verywell was around 12 years old so thatthat that brought me into ai and thenthe second part was later on uh when iwas around 1617 and i was writing gamesprofessionally designing games uhwriting a game called theme park whichum had ai as a core gameplay componentas part of the simulation um and it soldyou know millions of copies around theworld and people loved the way that theai even though it was relatively simpleby today's ai standards um was wasreacting to the way you as the playerplayed it so it was called a sandboxgame so it's one of the first types ofgames like that along with simcity andit meant that every game you played wasuniqueis there something you could say just ona small tangentaboutreally impressive ai from a game designhuman enjoyment perspectivereally impressive ai that you've seen ingames and maybe what does 

### Augementation

In [19]:
llm = ChatGoogleGenerativeAI(model= "gemini-2.5-flash")

In [20]:
prompt= PromptTemplate(template =""" You are Helpfull assistant
                       answer only the question from the provided context
                       {context}
                       question : {question}""",
                      input_variables = ['context','question'])

In [23]:
question = "is the topic artificial intelligence Discuss in these video, if yes then explian why ?"
retrive_docs = retriever.invoke(question)

In [25]:
retrive_docs

[Document(id='622558f3-be0a-4df8-aa70-3c72a73ce6b2', metadata={}, page_content="of stuff doyou think in order to solve intelligencewe also need to solve consciousnessalong the way do you think agi systemsneed to have consciousness in order tobetruly intelligent yeah we thought aboutthis a lot actually and um i think thatmy guess is that consciousness andintelligence are double dissociable soyou can have one without the other bothways and i think you can see that withconsciousness in that i think someanimals and pets if you have a pet dogor something like that you can see someof the higher animals and dolphinsthings like that are uh haveself-awareness and uh very sociable umseem to dream um you know those kinds ofa lot of the traits one would regard asbeing kind of conscious and self-awareum and but yet they're not that smartright uh so they're not that intelligentby by say iq standards or something likethat yeah it's also possible that ourunderstanding of intelligence is flawedlike put

In [26]:
context = "\n\n".join(doc.page_content for doc in retrive_docs)

In [27]:
context

"of stuff doyou think in order to solve intelligencewe also need to solve consciousnessalong the way do you think agi systemsneed to have consciousness in order tobetruly intelligent yeah we thought aboutthis a lot actually and um i think thatmy guess is that consciousness andintelligence are double dissociable soyou can have one without the other bothways and i think you can see that withconsciousness in that i think someanimals and pets if you have a pet dogor something like that you can see someof the higher animals and dolphinsthings like that are uh haveself-awareness and uh very sociable umseem to dream um you know those kinds ofa lot of the traits one would regard asbeing kind of conscious and self-awareum and but yet they're not that smartright uh so they're not that intelligentby by say iq standards or something likethat yeah it's also possible that ourunderstanding of intelligence is flawedlike putting an iq to it sure maybe thething that a dog can dois actually gone very far

In [28]:
final_prompt = prompt.invoke({"context":context,"question":question})

In [29]:
final_prompt

StringPromptValue(text=" You are Helpfull assistant\n                       answer only the question from the provided context\n                       of stuff doyou think in order to solve intelligencewe also need to solve consciousnessalong the way do you think agi systemsneed to have consciousness in order tobetruly intelligent yeah we thought aboutthis a lot actually and um i think thatmy guess is that consciousness andintelligence are double dissociable soyou can have one without the other bothways and i think you can see that withconsciousness in that i think someanimals and pets if you have a pet dogor something like that you can see someof the higher animals and dolphinsthings like that are uh haveself-awareness and uh very sociable umseem to dream um you know those kinds ofa lot of the traits one would regard asbeing kind of conscious and self-awareum and but yet they're not that smartright uh so they're not that intelligentby by say iq standards or something likethat yeah it'

### Generation

In [30]:
answer = llm.invoke(final_prompt)

In [32]:
answer.content

'Yes, the topic is Artificial Intelligence.\n\nThe discussion revolves around several concepts central to AI:\n*   **AGI (Artificial General Intelligence) systems:** The question of whether these systems need consciousness to be truly intelligent is directly addressed.\n*   **Intelligence:** The text explores what constitutes intelligence, how it relates to consciousness, and how to "solve" it (a goal for AI research).\n*   **Chatbots and the Turing Test:** It mentions early chatbots like Eliza and discusses the flaws of the Turing Test, both of which are foundational concepts in AI history and evaluation.\n*   **DeepMind:** The text specifically names DeepMind, a prominent AI research organization, and discusses its goal of "solving intelligence" and the various components (algorithms, data, hardware, software, human infrastructure) involved in achieving this.'

##### Using Chain

In [41]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [42]:
def format_docs(retrive_docs):
    context = "\n\n".join(doc.page_content for doc in retrive_docs)
    return context 

In [47]:
parallel_chain = RunnableParallel({
    'context':retriever|RunnableLambda(format_docs),
    'questions':RunnablePassthrough()
})  

In [48]:
parser = StrOutputParser()

In [58]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you summarize the video')